In [7]:
import json
import pathlib
import numpy as np
import torch

import descent.targets.energy

## Conventional energy/force targets from optimizations

In [15]:
def convert(json_file, with_forces=True):
    with open(json_file, "r") as f:
        contents = json.load(f)

    descent_entries = []

    for mapped_smiles, entries in contents.items():
        n_atoms = len(entries[0]["coords"]) // 3
        coords = []
        energy = []
        forces = []

        for conformer in entries:
            coords.append(
                np.array(conformer["coords"]).reshape((n_atoms, 3))
            )
            energy.append(conformer["energy"])
            if with_forces:
                forces.append(
                    np.array(conformer["forces"]).reshape((n_atoms, 3))
                )

        entry = {
            "smiles": mapped_smiles,
            "coords": torch.tensor(coords),
            "energy": torch.tensor(energy),
            "forces": torch.tensor(forces), # empty if missing
        }
        descent_entries.append(entry)
        
    dataset = descent.targets.energy.create_dataset(entries=descent_entries)
    output_directory = pathlib.Path(json_file.stem)
    dataset.save_to_disk(output_directory)
    print(f"Saved to {output_directory}")
    

In [16]:
json_files = pathlib.Path("jsons").glob("*.json")
for json_file in json_files:
    convert(json_file, with_forces=not "torsion" in str(json_file))

/Users/lily/micromamba/envs/smee-descent/lib/python3.11/site-packages/descent/targets/energy.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "coords": torch.tensor(entry["coords"]).flatten().tolist(),
/Users/lily/micromamba/envs/smee-descent/lib/python3.11/site-packages/descent/targets/energy.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "energy": torch.tensor(entry["energy"]).flatten().tolist(),
/Users/lily/micromamba/envs/smee-descent/lib/python3.11/site-packages/descent/targets/energy.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sou

Saved to original-torsiondrives
Saved to original-optimizations
Saved to combined-torsiondrives
Saved to combined-optimizations
Saved to additional-optimizations
Saved to additional-torsiondrives
